In [ ]:
#Here’s an example of addition of renewable generator to Bus 5:


In [ ]:
import pandas as pd
import pandapower as pp
import numpy as np

# Initialize network
net = pp.create_empty_network()

In [ ]:
# Create high voltage buses (11 kV)
bus1 = pp.create_bus(net, vn_kv=11, name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=11, name="Bus 2")

In [ ]:
# Create medium voltage buses (0.4 kV)
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3")
bus4 = pp.create_bus(net, vn_kv=0.4, name="Bus 4")

In [ ]:
# Low voltage buses connected to loads
bus5 = pp.create_bus(net, vn_kv=0.4, name="Bus 5")
bus6 = pp.create_bus(net, vn_kv=0.4, name="Bus 6")

In [ ]:
# External grid connections
pp.create_ext_grid(net, bus=bus1, vm_pu=1.0, name="Grid Connection 1")
pp.create_ext_grid(net, bus=bus2, vm_pu=1.0, name="Grid Connection 2")

In [ ]:
# Transformers with 11 kV to 0.4 kV ratio
pp.create_transformer_from_parameters(
    net, hv_bus=bus1, lv_bus=bus3, sn_mva=1, vn_hv_kv=11, vn_lv_kv=0.4, 
    vk_percent=6, vkr_percent=0.5, pfe_kw=0, i0_percent=0.1, name="Transformer 1"
)
pp.create_transformer_from_parameters(
    net, hv_bus=bus2, lv_bus=bus4, sn_mva=1, vn_hv_kv=11, vn_lv_kv=0.4, 
    vk_percent=6, vkr_percent=0.5, pfe_kw=0, i0_percent=0.1, name="Transformer 2"
)

In [ ]:
# High-voltage loads (connected to Bus 1 and Bus 2)
pp.create_load(net, bus=bus1, p_mw=0.8, q_mvar=0.4, name="HV Load 1")
pp.create_load(net, bus=bus2, p_mw=0.9, q_mvar=0.45, name="HV Load 2")

In [ ]:
# Low-voltage loads (connected to Bus 5 and Bus 6)
pp.create_load(net, bus=bus5, p_mw=0.4, q_mvar=0.2, name="LV Load 3")
pp.create_load(net, bus=bus6, p_mw=0.3, q_mvar=0.15, name="LV Load 4")

In [ ]:
# Lines between transformers and load buses
pp.create_line_from_parameters(
    net, from_bus=bus3, to_bus=bus5, length_km=0.1, r_ohm_per_km=0.1, 
    x_ohm_per_km=0.2, c_nf_per_km=0, max_i_ka=0.4, name="Line 1"
)
pp.create_line_from_parameters(
    net, from_bus=bus4, to_bus=bus6, length_km=0.1, r_ohm_per_km=0.1, 
    x_ohm_per_km=0.2, c_nf_per_km=0, max_i_ka=0.4, name="Line 2"
)

In [ ]:
# Add a solar PV generator to Bus 5
solar_pv = pp.create_sgen(net, bus=bus5, p_mw=0.5, q_mvar=0.0, name="Solar PV on Bus 5")

In [ ]:
# Define a load profile for 'Load on Bus 1'
time_steps = 10  # Example: 10 time steps
load_profile_bus1 = np.linspace(0.5, 1.5, time_steps)  # Linearly increasing load (0.5 MW to 1.5 MW)


In [ ]:
# Define a generation profile for Solar PV (simulating a daytime output pattern)
solar_profile = np.array([0.0, 0.2, 0.5, 0.7, 1.0, 0.8, 0.5, 0.3, 0.1, 0.0])  # Scaled to max generation

In [ ]:
# Run load flow for each time step and apply the load profile
for t in range(time_steps):
    # Update load on Bus 1 with the profile value for this time step
    net.load.at[0, "p_mw"] = load_profile_bus1[t]  # Load on Bus 1

    # Run load flow
    pp.runpp(net,numba=False)
    
    # Print results for this time step
    print(f"Time step {t + 1}:")
    print("Load on Bus 1 (MW):", load_profile_bus1[t])
    print("Bus voltages (pu):\n", net.res_bus.vm_pu)
    print("Solar PV Generation on Bus 5 (MW):", solar_profile[t])
    print("Line loading (%):\n", net.res_line.loading_percent)
    print("\n" + "-"*40 + "\n")

In [ ]:
# Switch to change topology between Bus 5 and Bus 6
switch = pp.create_switch(net, bus=bus5, element=bus6, et="b", closed=True, name="Topology Switch",numba=False)

# Function to run load flow with switch open and closed
def run_loadflow(switch_closed):
    net.switch.loc[switch, 'closed'] = switch_closed
    pp.runpp(net,numba=False)
    status = "closed" if switch_closed else "open"
    print(f"\n=== Load Flow Results with Switch {status} ===")
    print("Bus Voltages:\n", net.res_bus[["vm_pu", "va_degree"]])
    print("Transformer Loading:\n", net.res_trafo[["p_hv_mw", "q_hv_mvar", "p_lv_mw", "q_lv_mvar"]])
    print("Line Loadings:\n", net.res_line[["loading_percent"]])
    print("Load Power:\n", net.res_load[["p_mw", "q_mvar"]])


In [ ]:
# Run load flows
run_loadflow(switch_closed=True)  # Switch closed
run_loadflow(switch_closed=False) # Switch open